# **2023 NFL Big Data Bowl**
### Sol Ben-Ishay

## **Data Extraction/Wrangling**

### **Import libraries**

In [1]:
# Import libraries
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nfl_data_py as nfl
from scipy import stats 
from utils import animate_play

### **Read in the data**

In [2]:
# Read in the internal BDB competition data
games_df = pd.read_csv("../Data/games.csv")
plays_df = pd.read_csv("../Data/plays.csv")
players_df = pd.read_csv("../Data/players.csv")
scouting_df = pd.read_csv("../Data/pffScoutingData.csv")
tracking_df_list = ["../Data/tracking/week1.csv", "../Data/tracking/week2.csv", "../Data/tracking/week3.csv", "../Data/tracking/week4.csv",
"../Data/tracking/week5.csv", "../Data/tracking/week6.csv", "../Data/tracking/week7.csv", "../Data/tracking/week8.csv"]
tracking_df = pd.concat(map(pd.read_csv, tracking_df_list))

### **Create+clean the main data**

In [ ]:
# Get the play-level data
coi = ['gameId', 'season', 'week', 'homeTeamAbbr', 'visitorTeamAbbr', 'playId',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'penaltyYards',
       'prePenaltyPlayResult', 'playResult', 'foulName1', 'foulNFLId1',
       'foulName2', 'foulNFLId2', 'foulName3', 'foulNFLId3',
       'absoluteYardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInBox', 'personnelD', 'dropBackType', 'pff_playAction',
       'pff_passCoverage', 'pff_passCoverageType']
play_level_df = games_df.merge(plays_df, on='gameId', how='left')[coi]

# Replace n/a yardlineSide
play_level_df = play_level_df.fillna(value={'yardlineSide': 'MIDDLE'})

# Drop one play with null absolute yard line number
play_level_df = play_level_df[~((play_level_df.gameId == 2021091904) & (play_level_df.playId == 3676))]

# play_level_df.head(5)

In [ ]:
# Get the player-level data
coi = ['gameId', 'playId', 'nflId', 'displayName', 'officialPosition',
       'pff_role', 'pff_positionLinedUp', 'pff_hit', 'pff_hurry',
       'pff_sack', 'pff_beatenByDefender', 'pff_hitAllowed',
       'pff_hurryAllowed', 'pff_sackAllowed', 'pff_nflIdBlockedPlayer',
       'pff_blockType', 'pff_backFieldBlock']
player_level_df = (games_df.merge(plays_df, on='gameId', how='left')
                .merge(scouting_df, on=['gameId','playId'], how='left')
                .merge(players_df, on='nflId', how='left'))[coi]

# Get whether a defender got a pressure on the play
player_level_df['pff_pressure'] = player_level_df[['pff_hit','pff_hurry','pff_sack']].sum(axis=1)

# Separate the IDs of the OL/DL/QBs on the field for each play for easy use with the frame-level data
#1. Get the OL for each play
ol_on_play_df = player_level_df.query("officialPosition in ['C','G','T']")[coi]
#2. Get the DL for each play
dl_on_play_df = player_level_df.query("officialPosition in ['NT','DT','DE']")[coi]
#3. Get the QB for each play
qb_on_play_df = player_level_df.query("pff_role == 'Pass' and officialPosition == 'QB'")[coi]
                
# player_level_df.head(5)

In [ ]:
# Get the frame-level data
coi = ['gameId', 'playId', 'frameId', 'event', 'time', 'nflId', 'displayName', 'team', 'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir']
frame_level_df = (games_df.merge(plays_df, on='gameId', how='left')
                .merge(tracking_df, on=['gameId','playId'], how='left')
                .merge(players_df, on='nflId', how='left'))[coi]

# Convert dt_time column to time
frame_level_df['time'] = pd.to_datetime(frame_level_df['time'])

# Replace n/a values for ball rows   
frame_level_df = frame_level_df.fillna(value={"nflId":0,"displayName":"ball"})

# Separate the frame-level data of OL/DL/QBs for easy repeated use
coi = ['gameId', 'playId', 'nflId', 'frameId', 'x', 'y', 's', 'o', 'dir', 'playDirection']
# 1. Get the tracking data of the DL at every frame
dl_pos_df = dl_on_play_df.merge(frame_level_df, on=['gameId','playId','nflId'], how='left')[coi]
# 2. Get the tracking data of the OL/LOS blockers for each frame
ol_pos_df = ol_on_play_df.merge(frame_level_df, on=['gameId','playId','nflId'], how='left')[coi]
# 3. Get the tracking data of the QB for each frame
qb_pos_df = qb_on_play_df.merge(frame_level_df, on=['gameId','playId','nflId'], how='left')[coi]

# frame_level_df.head(5)

#### **Check for NAs**

In [ ]:
# Check for NA data
ok_na_cols = ["penaltyYards", "foulName1", "foulNFLId1", "foulName2", "foulNFLId2",
            "foulName3", "foulNFLId3", "offenseFormation", "personnelO", "defendersInBox",
            "personnelD", "dropBackType"]
if sum(play_level_df.loc[:,~play_level_df.columns.isin(ok_na_cols)].isna().sum().values) != 0:
    print(play_level_df.loc[:,~play_level_df.columns.isin(ok_na_cols)].isna().sum().loc[lambda x: x > 0])

In [ ]:
# Check for NA data
ok_na_cols = ["pff_hit", "pff_hurry", "pff_sack", "pff_beatenByDefender",
            "pff_hitAllowed", "pff_hurryAllowed", "pff_sackAllowed",
            "pff_nflIdBlockedPlayer", "pff_blockType", "pff_backFieldBlock"]
if sum(player_level_df.loc[:,~player_level_df.columns.isin(ok_na_cols)].isna().sum().values) != 0:
    print(player_level_df.isna().sum().loc[lambda x: x > 0])

In [ ]:
# Check for NA data
ok_na_cols = ['o','dir']
if sum(frame_level_df.loc[:,~frame_level_df.columns.isin(ok_na_cols)].isna().sum().values) != 0:
    print(frame_level_df.isna().sum().loc[lambda x: x > 0])

### **Create the secondary data**

#### **Main competition derived**

##### **Time of the snap and "action" for each play**

*Get the time of the snap and action event for each play for further use*

An action event is defined as the earliest of the following:  
  - A throw  
  - A non-throw, but play-ending event (QB sack, strip sack)
  - Any play where the QB doesn't scramble  
  - NOT YET: 3.5 seconds post-snap

A hard time cap is useful for reducing noise introduced by “broken” plays. Plays without a targeted receiver are not used for training, but are included for predictions so that players can be quantified on all pass plays.

In [ ]:
# 1. Get the time of all the key events for each play (all individual snap/action events)
potential_snap_events = ['ball_snap', 'autoevent_ballsnap']
potential_action_events = ['pass_forward', 'autoevent_passforward', 'qb_sack', 'fumble', 'run', 'qb_strip_sack', 'lateral', 'handoff', 'tackle']
conds = f"(event in {potential_snap_events}) or (event in {potential_action_events})"
coi = ['gameId', 'playId', 'event', 'frameId', 'time']
play_event_times_df = frame_level_df.query(conds)[coi].drop_duplicates()

# 2. Get the time of the snap and action event for each play
# Note: Not distinguishing snap/action times based on if they're from auto/manual events
# Utilizing the times of the earliest snap and earliest action event to calculate the snap to action time.
play_event_times_df.loc[play_event_times_df['event'].isin(potential_snap_events),'event'] = "ballsnap"
play_event_times_df.loc[play_event_times_df['event'].isin(potential_action_events),'event'] = "action"
play_event_times_df = play_event_times_df.groupby(['gameId','playId','event']).min().reset_index().sort_values(by=['time'])

# 3. Get the frame of the snap
coi = ['gameId','playId','frameId']
frame_of_snap_df = play_event_times_df.query("event == 'ballsnap'")[coi]

# 4. Get the frame of the action
coi = ['gameId','playId','frameId']
frame_of_action_df = play_event_times_df.query("event == 'action'")[coi]

#5. Merge with the play-level data
play_level_df = (play_level_df.merge(frame_of_snap_df.rename(columns={'frameId':'frameId_snap'}), how='left', on=['gameId','playId'])
                .merge(frame_of_action_df.rename(columns={'frameId':'frameId_action'}), how='left', on=['gameId','playId']))

##### **Time from the snap to "action" for each play**

*Get the time from snap to an action event for each play for further analyses*

In [ ]:
# 1. Drop rows where the tracking data is missing either a snap or action event
bad_rows = (play_event_times_df[['gameId','playId','event']]
            .groupby(['gameId','playId']).count().reset_index()
            .query('event != 2').drop(columns=['event']))
outer = play_event_times_df.merge(bad_rows, on=['gameId','playId'], how='outer', indicator=True)
play_event_times_df = outer[outer._merge == 'left_only'].drop(columns=['_merge'])

# 2. Get the snap to action event time for each play
coi = ['gameId', 'playId', 'time']
snap_to_action_df = play_event_times_df[coi].groupby(['gameId', 'playId']).agg(lambda x: np.max(x) - np.min(x)).reset_index().rename(columns={'time':'snap_to_action_time'})
snap_to_action_df['snap_to_action_time'] = snap_to_action_df['snap_to_action_time'].apply(lambda x: x.total_seconds())
snap_to_action_df = snap_to_action_df.sort_values(by=['snap_to_action_time']).reset_index(drop=True)
snap_to_action_df.head()

#3. Merge with the play-level data
play_level_df = play_level_df.merge(snap_to_action_df, how='left', on=['gameId','playId'])

##### **Players for each play separated by position**

In [ ]:
coi = ["gameId","playId","nflId"]
#1. Get the DL for each play
dl_on_play_df = player_level_df.query("officialPosition in ['NT','DT','DE']")[coi]
#2. Get the OL for each play
ol_on_play_df = player_level_df.query("officialPosition in ['C','G','T']")[coi]
#3. Get the QB for each play
qb_on_play_df = player_level_df.query("pff_role == 'Pass' and officialPosition == 'QB'")[coi]

##### **Tracking data for each frame separated by position**

In [ ]:
coi = ['gameId', 'playId', 'nflId', 'frameId', 'x', 'y', 's', 'o', 'dir', 'playDirection']
# 1. Get the tracking data of the DL at every frame
dl_pos_df = dl_on_play_df.merge(frame_level_df, on=['gameId','playId','nflId'], how='left')[coi]
# 2. Get the tracking data of the OL/LOS blockers for each frame
ol_pos_df = ol_on_play_df.merge(frame_level_df, on=['gameId','playId','nflId'], how='left')[coi]
# 3. Get the tracking data of the QB for each frame
qb_pos_df = qb_on_play_df.merge(frame_level_df, on=['gameId','playId','nflId'], how='left')[coi]